In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import FloatSlider, IntSlider
from IPython.display import display

sns.set(style="whitegrid")
np.random.seed(42)

# Utility function for plotting
def plot_sampling_distribution(population, sample_size, num_samples, pop_label="Population"):
    true_mean = np.mean(population)
    sample_means = [np.mean(np.random.choice(population, size=sample_size, replace=True)) 
                    for _ in range(num_samples)]

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(population, bins=50, kde=True, color='gray')
    plt.axvline(true_mean, color='red', linestyle='--', label=f"Mean = {true_mean:.2f}")
    plt.legend()
    plt.title(f"{pop_label} distribution")

    plt.subplot(1, 2, 2)
    sns.histplot(sample_means, bins=30, kde=True, color='royalblue')
    plt.axvline(true_mean, color='red', linestyle='--')
    plt.title(f"Sampling distribution (n={sample_size})")
    plt.tight_layout()
    plt.show()

# =================================================
# 1. SIZE (Normal distribution)
# =================================================
def simulate_size(mu, sigma, n, num_samples):
    population = np.random.normal(loc=mu, scale=sigma, size=100_000)
    plot_sampling_distribution(population, n, num_samples, pop_label="Size")

size_widget = widgets.interactive(
    simulate_size,
    mu=FloatSlider(min=150, max=200, step=1, value=175, description='Mean (cm)'),
    sigma=FloatSlider(min=1, max=20, step=1, value=10, description='Std dev'),
    n=IntSlider(min=2, max=300, step=1, value=30, description='Sample size'),
    num_samples=IntSlider(min=100, max=5000, step=100, value=1000, description='No. samples')
)
size_tab = widgets.VBox([widgets.HTML("<h3>📏 Body Size (Normal)</h3>"), size_widget])

# =================================================
# 2. INCOME (Lognormal)
# =================================================
def simulate_income(mean, sigma, n, num_samples):
    population = np.random.lognormal(mean=mean, sigma=sigma, size=100_000)
    plot_sampling_distribution(population, n, num_samples, pop_label="Income")

income_widget = widgets.interactive(
    simulate_income,
    mean=FloatSlider(min=5, max=12, step=0.5, value=10, description='Mean log'),
    sigma=FloatSlider(min=0.1, max=2, step=0.1, value=0.5, description='Sigma'),
    n=IntSlider(min=2, max=300, step=1, value=30, description='Sample size'),
    num_samples=IntSlider(min=100, max=5000, step=100, value=1000, description='No. samples')
)
income_tab = widgets.VBox([widgets.HTML("<h3>💰 Income (Lognormal)</h3>"), income_widget])

# =================================================
# 3. DICE (Uniform discrete)
# =================================================
def simulate_dice(n, num_samples):
    population = np.random.choice([1, 2, 3, 4, 5, 6], size=100_000, p=[1/6]*6)
    plot_sampling_distribution(population, n, num_samples, pop_label="Dice")

dice_widget = widgets.interactive(
    simulate_dice,
    n=IntSlider(min=2, max=300, step=1, value=30, description='Sample size'),
    num_samples=IntSlider(min=100, max=5000, step=100, value=1000, description='No. samples')
)
dice_tab = widgets.VBox([widgets.HTML("<h3>🎲 Dice Toss (Uniform)</h3>"), dice_widget])

# =================================================
# 4. COIN (Binomial)
# =================================================
def simulate_coin(p, n, num_samples):
    population = np.random.choice([0, 1], size=100_000, p=[1-p, p])
    plot_sampling_distribution(population, n, num_samples, pop_label="Coin toss (1 = heads)")

coin_widget = widgets.interactive(
    simulate_coin,
    p=FloatSlider(min=0, max=1, step=0.05, value=0.5, description='P(Heads)'),
    n=IntSlider(min=2, max=300, step=1, value=30, description='Sample size'),
    num_samples=IntSlider(min=100, max=5000, step=100, value=1000, description='No. samples')
)
coin_tab = widgets.VBox([widgets.HTML("<h3>🪙 Coin Toss (Binomial)</h3>"), coin_widget])

# =================================================
# 5. GRADES (Custom discrete)
# =================================================
grades = np.array([1, 2, 3, 4, 5])

def simulate_grades(p1, p2, p3, p4, p5, n, num_samples):
    p = np.array([p1, p2, p3, p4, p5])
    total = p.sum()
    if total == 0:
        p = np.ones_like(p) / len(p)
        print("⚠️ All probabilities were zero. Using uniform distribution.")
    elif not np.isclose(total, 1.0):
        p = p / total
        print(f"⚠️ Probabilities rescaled (original sum = {total:.2f})")

    population = np.random.choice(grades, size=100_000, p=p)
    plot_sampling_distribution(population, n, num_samples, pop_label="Grades")

grades_widget = widgets.interactive(
    simulate_grades,
    p1=FloatSlider(min=0, max=1, step=0.05, value=0.2, description='P(1)'),
    p2=FloatSlider(min=0, max=1, step=0.05, value=0.2, description='P(2)'),
    p3=FloatSlider(min=0, max=1, step=0.05, value=0.2, description='P(3)'),
    p4=FloatSlider(min=0, max=1, step=0.05, value=0.2, description='P(4)'),
    p5=FloatSlider(min=0, max=1, step=0.05, value=0.2, description='P(5)'),
    n=IntSlider(min=2, max=300, step=1, value=30, description='Sample size'),
    num_samples=IntSlider(min=100, max=5000, step=100, value=1000, description='No. samples')
)
grades_tab = widgets.VBox([widgets.HTML("<h3>🎓 Grades (Custom Discrete)</h3>"), grades_widget])

# =================================================
# TAB LAYOUT
# =================================================
tab = widgets.Tab()
tab.children = [size_tab, income_tab, dice_tab, coin_tab, grades_tab]
tab.set_title(0, 'Size')
tab.set_title(1, 'Income')
tab.set_title(2, 'Dice')
tab.set_title(3, 'Coin')
tab.set_title(4, 'Grades')

display(tab)
